In [3]:
import os
import json
import re
import sys
sys.path.append("..")
import requests
import pandas as pd
from time import time
import logging
from multiprocessing.pool import ThreadPool

In [4]:
scraped_data = pd.read_csv("../../data/data_from_scrapy.csv")
scraped_fic = scraped_data[scraped_data['fic_summary_table'].notna()]
scraped_fic["id"] = scraped_fic["court"] + "_" + scraped_fic["docket"]

# a version without na dockets
scraped = scraped_fic[scraped_fic['docket'].notna()]
len(scraped_data[scraped_data['court'].isna()]) #0

# both
len(scraped_data),len(scraped_fic),len(scraped)

/Users/yuchen.zhang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


(5771, 5254, 5249)

In [5]:
def countpdf(data):
    total = 0
    for x,y in withfic.iterrows():
        #save subtable
        table = pd.read_json(data["fic_summary_table"][x])
        total = total + len(table)
    return total

Exploratory

In [7]:
sumtable = pd.read_excel("../../data/summary_table_all.xlsx")
sumtable.drop('Unnamed: 0', axis = 1,inplace = True)
scraped_data = pd.read_csv("../../data/data_from_scrapy.csv")

In [8]:
sumtable.head(3)

,Filing Name,Filing Date,District Court,Exchange,Ticker
0,"The GEO Group, Inc.",07/07/2020,S.D. Florida,New York SE,GEO
1,Wirecard AG,07/07/2020,E.D. Pennsylvania,OTC-BB,WCAGY
2,China XD Plastics Company Limited,07/06/2020,S.D. New York,NASDAQ,CXDC


In [9]:
sumtable["Exchange"]

0            New York SE
1                 OTC-BB
2                 NASDAQ
3                 NASDAQ
4                 NASDAQ
              ...       
5766              NASDAQ
5767         New York SE
5768    Privately Traded
5769              NASDAQ
5770              NASDAQ
Name: Exchange, Length: 5771, dtype: object

In [10]:
scraped_data.dtypes

case_brief              object
case_name               object
case_status             object
class_period_end        object
class_period_start      object
company_market          object
court                   object
date_filed              object
date_of_last_review     object
docket                  object
fic_doc_links          float64
fic_links_list          object
fic_summary_table       object
headquarters            object
industry                object
judge                   object
market_status           object
plaintiffs              object
sector                  object
ticker_symbol           object
url                     object
dtype: object

In [11]:
scraped_data.isnull().sum()

case_brief                0
case_name                 0
case_status              26
class_period_end          9
class_period_start       11
company_market           61
court                     0
date_filed                0
date_of_last_review      26
docket                   23
fic_doc_links          5771
fic_links_list          517
fic_summary_table       517
headquarters              4
industry                  2
judge                   117
market_status            62
plaintiffs                3
sector                    2
ticker_symbol           297
url                       0
dtype: int64

In [13]:
#calculate duration
scraped_data["class_period_start"] = pd.to_datetime(scraped_data["class_period_start"])
scraped_data["class_period_end"] = pd.to_datetime(scraped_data["class_period_end"])
scraped_data["class_duration"] = scraped_data["class_period_end"] - scraped_data["class_period_start"]

In [14]:
#subset the data to features needed
data = scraped_data[["case_status",
                    "class_period_end",
                    "class_period_start",
                    "company_market",
                    "court",
                    "date_filed",
                    "headquarters",
                    "industry",
                    "judge",
                    "market_status",
                    "sector",
                    "class_duration"]]

In [15]:
data.groupby("case_status").count()

,class_period_end,class_period_start,company_market,court,date_filed,headquarters,industry,judge,market_status,sector,class_duration
case_status,,,,,,,,,,,
DISMISSED,2665,2663,2649,2669,2669,2666,2668,2597,2648,2668,2663
ONGOING,524,524,503,524,524,524,524,509,504,524,524
SETTLED,2547,2547,2532,2552,2552,2551,2551,2523,2531,2551,2547


In [16]:
data_input = data[data["case_status"]!="ONGOING"]

In [28]:
data_input.isnull().sum()

case_status            26
class_period_end        9
class_period_start     11
company_market         40
court                   0
date_filed              0
headquarters            4
industry                2
judge                 102
market_status          42
sector                  2
class_duration         11
dtype: int64

In [27]:
data_input[data_input["judge"].isnull()]

,case_status,class_period_end,class_period_start,company_market,court,date_filed,headquarters,industry,judge,market_status,sector,class_duration
23,DISMISSED,2020-06-17,2020-06-04,NASDAQ,D. Delaware,6/17/20,United States,Biotechnology & Drugs,NaN,Public (Listed),Healthcare,13 days
155,SETTLED,2019-11-19,2015-11-11,New York SE,D. Oregon,1/30/20,Australia,Money Center Banks,NaN,Public (Listed),Financial,1469 days
572,DISMISSED,2019-01-05,2016-04-27,NASDAQ,C.D. California,1/8/19,United States,Personal & Household Products,NaN,Public (Listed),Consumer Non-Cyclical,983 days
614,DISMISSED,2018-12-05,2018-10-15,New York SE,D. Colorado,12/5/18,United States,Software & Programming,NaN,Public (Listed),Technology,51 days
827,DISMISSED,2018-05-17,2018-04-09,New York SE,D. Delaware,5/17/18,United States,Communications Equipment,NaN,Public (Listed),Technology,38 days
...,...,...,...,...,...,...,...,...,...,...,...,...
2403,DISMISSED,2011-10-07,2011-08-16,NASDAQ,W.D. Wisconsin,10/7/11,United States,Software & Programming,NaN,Public (Listed),Technology,52 days
2409,DISMISSED,2011-08-05,2011-02-25,New York SE,S.D. New York,9/23/11,United States,Money Center Banks,NaN,Public (Listed),Financial,161 days
2411,SETTLED,2011-07-27,2011-04-15,NASDAQ,S.D. New York,9/9/11,France,Semiconductors,NaN,Public (Listed),Technology,103 days
2413,SETTLED,2010-12-10,2009-07-23,NASDAQ,D. New Jersey,9/9/11,United States,Beverages (Non-Alcoholic),NaN,Public (Listed),Consumer Non-Cyclical,505 days
